<a href="https://colab.research.google.com/github/MehrdadJalali-AI/socialNetworkOptimization/blob/main/SOCAIL_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cupy as cp  # Replace numpy with cupy for GPU acceleration
import numpy as np  # For CPU-side operations and compatibility
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

# --- Configuration (Hyperparameters) ---
class Config:
    DIM = 30
    NUM_NODES = 100
    ITERATIONS = 2000
    NUM_RUNS = 100

    K = 4
    P = 0.3

    ALPHA = 0.4
    BETA = 0.4
    GAMMA = 0.2
    DELTA = 0.2

    MUTATION_RATE = 0.1
    MUTATION_STRENGTH_BASE = 0.1
    MUTATION_STRENGTH_MIN = 0.01

# --- Benchmark Functions (GPU-Compatible) ---
def sphere(x): return cp.sum(x**2)
def schwefel_2_22(x): return cp.sum(cp.abs(x)) + cp.prod(cp.abs(x))
def schwefel_1_2(x): return cp.sum(cp.array([cp.sum(x[:i+1])**2 for i in range(len(x))]))
def schwefel_2_21(x): return cp.max(cp.abs(x))
def rosenbrock(x): return cp.sum(cp.array([100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2 for i in range(len(x)-1)]))
def step(x): return cp.sum(cp.floor(x + 0.5)**2)
def quartic(x): return cp.sum(cp.array([(i+1) * xi**4 for i, xi in enumerate(x)])) + cp.random.uniform(0, 1)
def schwefel_2_26(x): return 418.9829 * len(x) - cp.sum(x * cp.sin(cp.sqrt(cp.abs(x))))
def rastrigin(x): return 10 * len(x) + cp.sum(x**2 - 10 * cp.cos(2 * cp.pi * x))
def ackley(x):
    a, b, c = 20, 0.2, 2 * cp.pi
    d = len(x)
    return -a * cp.exp(-b * cp.sqrt(cp.sum(x**2) / d)) - cp.exp(cp.sum(cp.cos(c * x)) / d) + a + cp.e
def griewank(x): return cp.sum(x**2) / 4000 - cp.prod(cp.cos(x / cp.sqrt(cp.arange(1, len(x)+1)))) + 1
def penalized(x):
    y = 1 + (x + 1) / 4
    term1 = 10 * cp.sin(cp.pi * y[0])**2
    term2 = cp.sum(cp.array([(y[i] - 1)**2 * (1 + 10 * cp.sin(cp.pi * y[i+1])**2) for i in range(len(x)-1)]))
    term3 = (y[-1] - 1)**2
    u = cp.sum(cp.array([100 * (xi - 10)**4 if xi > 10 else (-10 - xi)**4 if xi < -10 else 0 for xi in x]))
    return (cp.pi / len(x)) * (term1 + term2 + term3) + u
def penalized2(x):
    term1 = cp.sin(3 * cp.pi * x[0])**2
    term2 = cp.sum(cp.array([(x[i] - 1)**2 * (1 + cp.sin(3 * cp.pi * x[i+1])**2) for i in range(len(x)-1)]))
    term3 = (x[-1] - 1)**2 * (1 + cp.sin(2 * cp.pi * x[-1])**2)
    u = cp.sum(cp.array([0.1 * (xi - 5)**4 if xi > 5 else (-5 - xi)**4 if xi < -5 else 0 for xi in x]))
    return 0.1 * (term1 + term2 + term3) + u

# --- Function List and Bounds ---
FUNCTIONS = {
    'Sphere': (sphere, [-100, 100]),
    'Schwefel_2_22': (schwefel_2_22, [-10, 10]),
    'Schwefel_1_2': (schwefel_1_2, [-100, 100]),
    'Schwefel_2_21': (schwefel_2_21, [-100, 100]),
    'Rosenbrock': (rosenbrock, [-30, 30]),
    'Step': (step, [-100, 100]),
    'Quartic': (quartic, [-1.28, 1.28]),
    'Schwefel_2_26': (schwefel_2_26, [-500, 500]),
    'Rastrigin': (rastrigin, [-5.12, 5.12]),
    'Ackley': (ackley, [-32, 32]),
    'Griewank': (griewank, [-600, 600]),
    'Penalized': (penalized, [-50, 50]),
    'Penalized2': (penalized2, [-50, 50])
}

# --- Initialize Graph and Population (CPU for graph, GPU for positions) ---
def initialize_population(num_nodes, dim, bounds):
    G = nx.watts_strogatz_graph(num_nodes, k=Config.K, p=Config.P)  # CPU-based
    positions = cp.random.uniform(bounds[0], bounds[1], (num_nodes, dim))  # GPU array
    for i, node in enumerate(G.nodes):
        G.nodes[node]['position'] = positions[i]
        G.nodes[node]['fitness'] = None
    return G, positions

# --- Evaluate Fitness (GPU) ---
def evaluate_fitness(G, positions, func):
    fitness_values = cp.zeros(len(G.nodes))
    for i, node in enumerate(G.nodes):
        pos = positions[i]
        fitness_values[i] = func(pos)
        G.nodes[node]['fitness'] = float(fitness_values[i].get())  # Store on CPU for NetworkX
    return fitness_values

# --- Diffusion with Centrality and Mutation (GPU) ---
def diffuse(G, positions, gbest_pos, elite_pos, config, bounds, iteration, max_iterations, enable_mutation=True):
    centrality = nx.betweenness_centrality(G)  # CPU-based
    centrality_array = cp.array([centrality[n] for n in G.nodes])
    fitness_values = cp.array([G.nodes[n]['fitness'] for n in G.nodes])
    influence = 1 - (fitness_values / (cp.max(fitness_values) + 1e-6))

    alpha_t = config.ALPHA * (1 - iteration / max_iterations)
    beta_t = config.BETA * (1 - iteration / max_iterations)
    gamma_t = config.GAMMA * (iteration / max_iterations)
    delta_t = config.DELTA * (iteration / max_iterations)

    new_positions = cp.zeros_like(positions)
    for node in G.nodes:
        neighbors = list(G.neighbors(node))
        if not neighbors:
            new_positions[node] = positions[node]
            continue

        weights = cp.array([alpha_t * centrality[n] + beta_t * influence[i] for i, n in enumerate(neighbors)])
        weights = weights / cp.sum(weights) if cp.sum(weights) > 0 else cp.ones_like(weights) / len(weights)
        neighbor_positions = positions[neighbors]
        neighbor_contribution = cp.average(neighbor_positions, weights=weights, axis=0)

        new_pos = (
            positions[node] * (1 - alpha_t - beta_t - gamma_t - delta_t) +
            neighbor_contribution * (alpha_t + beta_t) +
            gbest_pos * gamma_t +
            elite_pos * delta_t
        )
        new_positions[node] = cp.clip(new_pos, bounds[0], bounds[1])

    if enable_mutation:
        mutation_strength = config.MUTATION_STRENGTH_BASE * (1 - iteration / max_iterations) + config.MUTATION_STRENGTH_MIN
        mutation_range = mutation_strength * (bounds[1] - bounds[0])
        median_fitness = cp.median(fitness_values)
        mask = (fitness_values > median_fitness) & (cp.random.rand(len(G.nodes)) < config.MUTATION_RATE)

        perturb = cp.random.uniform(-mutation_range, mutation_range, positions.shape)
        new_positions = cp.where(mask[:, cp.newaxis], cp.clip(new_positions + perturb, bounds[0], bounds[1]), new_positions)

    for i, node in enumerate(G.nodes):
        positions[i] = new_positions[i]

    return positions

# --- SOCIAL Optimizer (GPU) ---
def social_optimize(func_name, config=Config(), enable_mutation=True):
    func, bounds = FUNCTIONS[func_name]
    G, positions = initialize_population(config.NUM_NODES, config.DIM, bounds)

    fitness_values = evaluate_fitness(G, positions, func)
    gbest_idx = cp.argmin(fitness_values)
    gbest_pos = positions[gbest_idx].copy()
    gbest_fitness = fitness_values[gbest_idx]
    elite_pos = gbest_pos.copy()
    elite_fitness = float(gbest_fitness.get())

    fitness_history = []

    for iteration in range(config.ITERATIONS):
        fitness_values = evaluate_fitness(G, positions, func)
        min_idx = cp.argmin(fitness_values)
        if fitness_values[min_idx] < gbest_fitness:
            gbest_pos = positions[min_idx].copy()
            gbest_fitness = fitness_values[min_idx]

        if float(gbest_fitness.get()) < elite_fitness:
            elite_pos = gbest_pos.copy()
            elite_fitness = float(gbest_fitness.get())

        fitness_history.append(elite_fitness)
        positions = diffuse(G, positions, gbest_pos, elite_pos, config, bounds, iteration, config.ITERATIONS, enable_mutation)

    return np.array(fitness_history), elite_pos.get(), elite_fitness, G

# --- Run, Compute Metrics, and Save to CSV ---
def run_all_functions(config=Config()):
    results = []

    for name, (func, bounds) in FUNCTIONS.items():
        best_fits = []
        all_histories = []
        final_populations = []

        for _ in range(config.NUM_RUNS):
            history, _, best_fit, G = social_optimize(name, config)
            best_fits.append(best_fit)
            all_histories.append(history)
            final_populations.append(G)

        # Compute metrics (on CPU for simplicity)
        best_fits = np.array(best_fits)
        mean_fit = np.mean(best_fits)
        std_fit = np.std(best_fits)
        robustness = std_fit**2
        diversity = np.mean([np.mean([np.std([G.nodes[n]['position'][i].get() for n in G.nodes]) for i in range(config.DIM)]) for G in final_populations])
        convergence_speed = next((i for i, fit in enumerate(all_histories[0]) if fit < mean_fit + std_fit), config.ITERATIONS)

        results.append({
            "Function": name,
            "Best Fitness": np.min(best_fits),
            "Worst Fitness": np.max(best_fits),
            "Mean Fitness": mean_fit,
            "Std Dev": std_fit,
            "Robustness": robustness,
            "Diversity": diversity,
            "Convergence Speed": convergence_speed
        })
        print(f"{name} - Best: {np.min(best_fits):.6f}, Mean: {mean_fit:.6f}, Std Dev: {std_fit:.6f}")

    # Save to CSV
    df = pd.DataFrame(results)
    df.to_csv('social_results.csv', index=False)
    print("\nResults saved to 'social_results.csv'")

    # Plot convergence
    plt.figure(figsize=(12, 8))
    for name in FUNCTIONS.keys():
        history = next(r["Mean Fitness"] for r in results if r["Function"] == name)
        plt.plot(all_histories[0], label=name)
    plt.xlabel("Iteration")
    plt.ylabel("Best Fitness")
    plt.title("SOCIAL Optimizer on Benchmark Functions (F1-F13)")
    plt.yscale('log')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return results

# --- Execute ---
if __name__ == "__main__":
    config = Config()
    results = run_all_functions(config)